In [ ]:
import os # Configure which GPU
if os.getenv("CUDA_VISIBLE_DEVICES") is None:
    gpu_num = 0 # Use "" to use the CPU
    os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_num}"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Import Sionna
try:
    import sionna
except ImportError as e:
    # Install Sionna if package is not already installed
    import os
    os.system("pip install sionna")
    import sionna

# Configure the notebook to use only a single GPU and allocate only as much memory as needed
# For more details, see https://www.tensorflow.org/guide/gpu
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)

# Avoid warnings from TensorFlow
tf.get_logger().setLevel('ERROR')

# Colab does currently not support the latest version of ipython.
# Thus, the preview does not work in Colab. However, whenever possible we
# strongly recommend to use the scene preview mode.
try: # detect if the notebook runs in Colab
    import google.colab
    no_preview = True # deactivate preview
except:
    if os.getenv("SIONNA_NO_PREVIEW"):
        no_preview = True
    else:
        no_preview = False

resolution = [480,320] # increase for higher quality of renderings

# Define magic cell command to skip a cell if needed
from IPython.core.magic import register_cell_magic
from IPython import get_ipython

@register_cell_magic
def skip_if(line, cell):
    if eval(line):
        return
    get_ipython().run_cell(cell)

# Set random seed for reproducibility
sionna.config.seed = 40

import matplotlib.pyplot as plt
import numpy as np

from sionna.rt import load_scene, PlanarArray, Transmitter, Receiver, Camera
from sionna.rt.utils import r_hat
from sionna.ofdm import ResourceGrid
from sionna.channel.utils import subcarrier_frequencies, cir_to_ofdm_channel
from sionna.constants import SPEED_OF_LIGHT

In [ ]:
scene = load_scene("../models/car2/car2.xml")

# Configure arrays for all transmitters and receivers in the scene
scene.tx_array = PlanarArray(1,1,0.5,0.5,"iso","V")
scene.rx_array = scene.tx_array
# Add a transmitter and a receiversudo /usr/bin/vmhgfs-fuse .host:/ /mnt/hgfs -o allow_other -o uid=1000 -o gid=1000 -o umask=022
scene.add(Transmitter("tx", [ 0.5, 0.3, 0.6]))
scene.add(Receiver("rx",    [ 1.1, 0.4, 0.5]))

# Compute paths
scene.preview()

In [ ]:
paths = scene.compute_paths(max_depth=3,
                            num_samples=1e6)
scene.preview(paths, show_devices=True, show_paths=True)
# tau表示每个信道的时延
print(paths.tau.numpy().squeeze())
print(paths.phi_t.numpy().squeeze() * 180 / 3.1415926)
# print(paths.tau.numpy().squeeze() * 3e8)
# print(paths.a.numpy().squeeze())
# a表示信道系数，a的幅值的平方表示接收信号的能量
# print(np.abs(paths.a.numpy().squeeze())**2)
# 用分贝形式表示
print(10 * np.log10(np.abs(paths.a.numpy().squeeze())**2))
# 每个物体经过的路径
# 0 - 木头门 | 1 - 玻璃窗 | 2 - 金属 | 3 - 墙面
print(paths.objects.numpy().squeeze())

# plt.figure()
# plt.stem(paths.tau.numpy().squeeze(), 10 * np.log10(np.abs(paths.a.numpy().squeeze())**2))
plt.figure()
plt.scatter(paths.tau.numpy().squeeze(), (paths.phi_t.numpy().squeeze() * 180 / 3.1415926)
, c=10 * np.log10(np.abs(paths.a.numpy().squeeze())**2), cmap='jet', s=200, edgecolors='k')
plt.colorbar(label="Power (dB)")
plt.xlabel("Delay (ns)")
plt.ylabel("Angle (°)")
plt.title("Power-Angle-Delay Profile (PADP)")
plt.grid(True)
plt.xlim([-0.3e-8, 6.3e-8])
plt.savefig("Result.svg")
plt.savefig("Result.png")
plt.show()